# Quantum Process Tomography

* **Author:** Christopher J. Wood (cjwood@us.ibm.com)
* **Last Updated:** November 8, 2018 

In [1]:
# Needed for functions
import numpy as np
import time

# Import QISKit classes
import qiskit
from qiskit import QuantumRegister, QuantumCircuit, Aer
from qiskit.tools.qi.qi import state_fidelity, outer

# Tomography functions
import sys, os
sys.path.append(os.path.abspath(os.path.join('../qiskit_ignis')))
import tomography as tomo

## 1-qubit process tomography example

In [2]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('Zp',), ('X',)): {'0': 2000},
 (('Zp',), ('Y',)): {'0': 995, '1': 1005},
 (('Zp',), ('Z',)): {'0': 968, '1': 1032},
 (('Zm',), ('X',)): {'1': 2000},
 (('Zm',), ('Y',)): {'0': 967, '1': 1033},
 (('Zm',), ('Z',)): {'0': 958, '1': 1042},
 (('Xp',), ('X',)): {'0': 1014, '1': 986},
 (('Xp',), ('Y',)): {'0': 992, '1': 1008},
 (('Xp',), ('Z',)): {'0': 2000},
 (('Yp',), ('X',)): {'0': 986, '1': 1014},
 (('Yp',), ('Y',)): {'1': 2000},
 (('Yp',), ('Z',)): {'0': 959, '1': 1041}}

In [3]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.fitters.cvx_fit(data, basis, PSD=True, trace_preserving=True, trace=2)
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.19025516510009766
fit fidelity: 0.9937142233768951

CVXOPT Fitter
fit time: 0.21304011344909668
fit fidelity: 0.9998838322013748


## 2-Qubit entangling circuit

In [4]:
# Bell-state entangling circuit
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.h(q[0])
circ.cx(q[0], q[1])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)

# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts)


t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_mle / 4))

t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)
print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 4, choi_cvx / 4))

MLE Fitter
fit time: 0.03369331359863281
fit fidelity: 0.9789523948844063

CVXOPT Fitter
fit time: 1.3920791149139404
fit fidelity: 0.999869582566958


## Using SIC-POVM preparation basis

In [6]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_data(circ).get('unitary')
# convert to Choi-matrix in column-major convention
choi_ideal = outer(ideal_unitary.ravel(order='F'))

# Generate process tomography circuits and run on qasm simulator
qpt_circs = tomo.process_tomography_circuits(circ, q, prep_labels='SIC', prep_basis='SIC')
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_counts = tomo.tomography_data(job.result(), qpt_circs)
qpt_counts

{(('S0',), ('X',)): {'0': 2000},
 (('S0',), ('Y',)): {'0': 1014, '1': 986},
 (('S0',), ('Z',)): {'0': 995, '1': 1005},
 (('S1',), ('X',)): {'0': 637, '1': 1363},
 (('S1',), ('Y',)): {'0': 965, '1': 1035},
 (('S1',), ('Z',)): {'0': 1946, '1': 54},
 (('S2',), ('X',)): {'0': 652, '1': 1348},
 (('S2',), ('Y',)): {'0': 1841, '1': 159},
 (('S2',), ('Z',)): {'0': 535, '1': 1465},
 (('S3',), ('X',)): {'0': 667, '1': 1333},
 (('S3',), ('Y',)): {'0': 195, '1': 1805},
 (('S3',), ('Z',)): {'0': 538, '1': 1462}}

In [7]:
# Extract fitter data from tomography counts
data, basis = tomo.fitter_data(qpt_counts, prep_basis='SIC')

# MLE Least-Squares tomographic reconstruction
t = time.time()
choi_mle = tomo.process_mle_fit(data, basis)
print('MLE Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_mle / 2))

# CVXOPT Semidefinite-Program tomographic reconstruction
t = time.time()
choi_cvx = tomo.process_cvx_fit(data, basis)

print('\nCVXOPT Fitter')
print('fit time:', time.time() - t)
print('fit fidelity:', state_fidelity(choi_ideal / 2, choi_cvx / 2))

MLE Fitter
fit time: 0.0012753009796142578
fit fidelity: 0.9969257435314389

CVXOPT Fitter
fit time: 0.08712172508239746
fit fidelity: 0.9998851567089635
